In [1]:
!pip list

Package            Version
------------------ --------
appnope            0.1.4
asttokens          2.4.1
certifi            2024.2.2
chardet            4.0.0
comm               0.2.2
debugpy            1.8.1
decorator          5.1.1
exceptiongroup     1.2.0
executing          2.0.1
filelock           3.14.0
fsspec             2024.3.1
idna               2.10
importlib_metadata 7.1.0
ipykernel          6.29.3
ipython            8.22.2
jedi               0.19.1
Jinja2             3.1.4
jupyter_client     8.6.1
jupyter_core       5.7.2
MarkupSafe         2.1.5
matplotlib-inline  0.1.7
mpmath             1.3.0
nest_asyncio       1.6.0
networkx           3.3
numpy              1.26.4
packaging          24.0
parso              0.8.4
pexpect            4.9.0
pickleshare        0.7.5
pillow             10.3.0
pip                23.3.1
platformdirs       4.2.1
prompt-toolkit     3.0.42
psutil             5.9.8
ptyprocess         0.7.0
pure-eval          0.2.2
Pygments           2.18.0
python-da

In [2]:
import os
import numpy as np
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
from torch.utils.data import random_split

import torchattacks

from tqdm import tqdm
import math
import random
import csv
import torchvision.transforms as transforms
from PIL import Image

## Set Seeds

In [3]:
def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Datasets class

In [4]:
class Cifar100(Dataset):
    def __init__(self, data, labels):
        super(Cifar100, self).__init__()
        self.data = data
        self.labels = labels
        self.datasize = len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


    def __len__(self):
        return self.datasize

## Configs

In [5]:
config = {
    "batch_size": 1,
    "model_type": "ensemble",
    "attack": "FGSM",
    "data_filepath": "./cifar-100_eval",
    "current_dir": "./",
    "save_dir": "adv_imgs",
    "output_filepath": "./adv_imgs",
    "steps": 90,
    "epsilon": 4 / 255,
    "decay": 0.9,
    "seeds": 10901036
}
config["alpha"] = config["epsilon"] / config["steps"]

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Load Datasets

In [6]:
same_seeds(config["seeds"])
eval_files = sorted([os.path.join(config["data_filepath"],x) for x in os.listdir(config["data_filepath"]) if x.endswith(".png")])
eval_imgs = torch.stack([transforms.ToTensor()(Image.open(filename)) for filename in eval_files])
eval_labels = torch.tensor([int(filename.split('/')[-1].split('_')[0]) for filename in eval_files], dtype=torch.long)
target_labels = torch.tensor([0]*len(eval_imgs), dtype=torch.long)

## Model(s)

In [7]:
from pytorchcv.model_provider import get_model as ptcv_get_model
class Ensemble(nn.Module):
    def __init__(self, model_names):
        super(Ensemble, self).__init__()
        self.models = nn.ModuleList([ptcv_get_model(model_name, pretrained=True) for model_name in model_names])

    def forward(self, x):
        y = [model(x) for model in self.models]
        y = torch.stack(y, dim=1).mean(dim=1)
        return y

In [8]:
training_models = ["resnet110_cifar100", "preresnet164bn_cifar100", "seresnet110_cifar100", "densenet40_k36_bc_cifar100", "diaresnet110_cifar100"]
# training_models = ["resnet110_cifar100", "preresnet20bn_cifar100", "seresnet20_cifar100"]
# victim_models = ["resnet110_cifar100", "preresnet164bn_cifar100", "seresnet110_cifar100"]
# victim_models = ["resnet110_cifar100", "preresnet164bn_cifar100", "seresnet110_cifar100", "densenet40_k36_bc_cifar100"]
victim_models = ["resnet110_cifar100", "preresnet164bn_cifar100", "seresnet110_cifar100", "densenet40_k36_bc_cifar100", "diaresnet164bn_cifar100"]

## Attack

In [9]:
if config["model_type"] == "ensemble":
    model_names = training_models
    with open("attack_info.txt", 'w') as f:
        f.write(f"Attack: DIFGSM \nEpsilon: {config['epsilon']}\nModel_type: {config['model_type']}\nDecay: {config['decay']}\nSteps: {config['steps']}\nModel_type: {config['model_type']}\nModel: {model_names}\n")
    ensemble_model = Ensemble(model_names).to(device)
    # attack = torchattacks.DIFGSM(ensemble_model, eps=config["epsilon"], decay=config["decay"], alpha=config["alpha"], steps=config["steps"])
    # attack = torchattacks.FGSM(ensemble_model, eps=config["epsilon"])
    attack = torchattacks.PGD(ensemble_model, eps=config["epsilon"], alpha=config["alpha"], steps=config["steps"])
    attack.set_mode_targeted_by_label(quiet=True)
    print("start attacking...")
    
    adv_images = attack(eval_imgs, target_labels)
    print("done")

if adv_images == None:
    print("No model found.")
    exit()

start attacking...


KeyboardInterrupt: 

## Save images

In [ ]:
if not os.path.exists(config["save_dir"]):
    os.mkdir(config["save_dir"])
for adv_image, eval_file in zip(adv_images, eval_files):
    save_image(adv_image, os.path.join(config["current_dir"], config["save_dir"], eval_file.split('/')[-1]))

## Test Accuracies

In [ ]:
adv_files = sorted([os.path.join(config["output_filepath"], x) for x in os.listdir(config["output_filepath"]) if x.endswith(".png")])
adv_images = torch.stack([transforms.ToTensor()(Image.open(filename)) for filename in adv_files])
adv_labels = torch.tensor([int(filename.split('/')[-1].split('_')[0]) for filename in adv_files], dtype=torch.long)

# eval_datasets = Cifar100(eval_imgs, eval_labels)
adv_datasets  = Cifar100(adv_images, adv_labels)

# eval_loader = DataLoader(eval_datasets, batch_size=config['batch_size'], shuffle=False, pin_memory=True)
adv_loader  = DataLoader(adv_datasets , batch_size=config['batch_size'], shuffle=False, pin_memory=True)

In [ ]:
test_model_names = victim_models
test_model = Ensemble(test_model_names)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    target = 0
    for img, _ in tqdm(test_loader):
        img = img.to(device)
        with torch.no_grad():
            pred = model(img)
            if torch.argmax(pred) == 0:
                target += 1
    return target / 500

# original_acc = predict(eval_loader, test_model, device)
adv_acc = predict(adv_loader, test_model, device)
# print(f"orig_images targeted attack success rate: {original_acc}")
print(f"adv_images targeted attack success rate: {adv_acc}")
# print("")
# with open("attack_info.txt", 'a') as f:
    # f.write(f"\nTestModel: {test_model_names[idx]}\nOriginal images accuracy: {original_acc}\nAdversarial images accuracy: {adv_acc}\n")

100%|██████████| 500/500 [01:10<00:00,  7.08it/s]

adv_images targeted attack success rate: 0.026


In [ ]:
def checkOutput(adv_images, eval_images):
    # check if adv_img 0~255
    assert torch.max(adv_images) <= 1
    assert torch.min(adv_images) >= 0
    # check if adv_img are in the range of epsilon from eval_img
    diff = torch.abs(adv_images - eval_images)
    print( torch.max(diff))
    print(config["epsilon"])